In [ ]:
!pip install pyspark
!pip3 install pyspark
!pip install findspark
!pip3 install findspark

In [1]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import *
spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.format("csv") \
.option("header", "true") \
.option("nullValue","NA") \
.option("inferSchema", "true").load("2007.csv")

In [3]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)

In [4]:
print("Partitions:", df.rdd.getNumPartitions())

Partitions: 6


In [5]:
#Eliminar columnes
df2 = df.drop("Flightfrom pyspark.sql.functions import exprNum","TailNum","UniqueCarrier")
#New frame with columns
df2 = df.select("Origin", "Dest", "ArrDelay", "DepDelay")
df2.show()
#Eliminar files on hi ha NA
df3 = df2.na.drop()

+------+----+--------+--------+
|Origin|Dest|ArrDelay|DepDelay|
+------+----+--------+--------+
|   SMF| ONT|       1|       7|
|   SMF| PDX|       8|      13|
|   SMF| PDX|      34|      36|
|   SMF| PDX|      26|      30|
|   SMF| PDX|      -3|       1|
|   SMF| PDX|       3|      10|
|   SMF| PHX|      47|      56|
|   SMF| PHX|      -2|       9|
|   SMF| PHX|      44|      47|
|   SMF| PHX|      -7|       3|
|   SMF| PHX|     -11|       1|
|   SMF| PHX|      52|      52|
|   SMF| SAN|      45|      53|
|   SMF| SAN|     -17|      -5|
|   SMF| SAN|      -5|       6|
|   SMF| SAN|      33|      44|
|   SMF| SAN|      -9|       0|
|   SMF| SAN|      -7|       2|
|   SMF| SAN|     -11|       1|
|   SMF| SAN|      36|      29|
+------+----+--------+--------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import expr
##Suma, expr we can do expressions
df4 = df3.withColumn("SumDelay", expr("ArrDelay + DepDelay"))
#Action, no transformation
df4.select("DepDelay", "ArrDelay", "SumDelay").show(10)

+--------+--------+--------+
|DepDelay|ArrDelay|SumDelay|
+--------+--------+--------+
|       7|       1|       8|
|      13|       8|      21|
|      36|      34|      70|
|      30|      26|      56|
|       1|      -3|      -2|
|      10|       3|      13|
|      56|      47|     103|
|       9|      -2|       7|
|      47|      44|      91|
|       3|      -7|      -4|
+--------+--------+--------+
only showing top 10 rows



In [7]:
from pyspark.sql.functions import max, min
#es recorre tot el df
df4.select(max("SumDelay"),min("SumDelay")).show()

+-------------+-------------+
|max(SumDelay)|min(SumDelay)|
+-------------+-------------+
|         5199|         -617|
+-------------+-------------+



In [8]:
from pyspark.sql.functions import avg
df4.select(max("SumDelay"),min("SumDelay"),avg("SumDelay")).show()

+-------------+-------------+-----------------+
|max(SumDelay)|min(SumDelay)|    avg(SumDelay)|
+-------------+-------------+-----------------+
|         5199|         -617|21.55425998256014|
+-------------+-------------+-----------------+



In [9]:
#temporal save in cache or ram
df4.cache()

DataFrame[Origin: string, Dest: string, ArrDelay: int, DepDelay: int, SumDelay: int]

In [10]:
df5 = df4.where("SumDelay < 0")
df5.show() #accio, nomes mostra els primer elements
df3.count()
df5.count()

+------+----+--------+--------+--------+
|Origin|Dest|ArrDelay|DepDelay|SumDelay|
+------+----+--------+--------+--------+
|   SMF| PDX|      -3|       1|      -2|
|   SMF| PHX|      -7|       3|      -4|
|   SMF| PHX|     -11|       1|     -10|
|   SMF| SAN|     -17|      -5|     -22|
|   SMF| SAN|      -9|       0|      -9|
|   SMF| SAN|      -7|       2|      -5|
|   SMF| SAN|     -11|       1|     -10|
|   SMF| SAN|      -6|       3|      -3|
|   SMF| SAN|     -14|       0|     -14|
|   SMF| SAN|      -9|      -5|     -14|
|   SMF| SNA|      -4|       0|      -4|
|   SMF| SNA|      -8|       2|      -6|
|   SMF| SNA|     -16|      -4|     -20|
|   SMF| SNA|      -7|       0|      -7|
|   SMF| SNA|     -15|      -4|     -19|
|   SNA| MDW|     -18|       0|     -18|
|   SNA| OAK|      -1|       0|      -1|
|   SNA| OAK|       0|      -1|      -1|
|   SNA| OAK|      -2|       0|      -2|
|   SNA| OAK|      -1|       0|      -1|
+------+----+--------+--------+--------+
only showing top

3676937

In [11]:
df5 = df4.where("SumDelay < 0").where("Origin == 'JFK'")
df5.show()
df5.count()

+------+----+--------+--------+--------+
|Origin|Dest|ArrDelay|DepDelay|SumDelay|
+------+----+--------+--------+--------+
|   JFK| CLE|     -13|     -19|     -32|
|   JFK| CLE|     -25|      -3|     -28|
|   JFK| CLE|     -25|      -8|     -33|
|   JFK| CLE|       4|     -14|     -10|
|   JFK| CLE|     -17|      -2|     -19|
|   JFK| CLE|      -4|      -3|      -7|
|   JFK| CLE|      -4|      -3|      -7|
|   JFK| CLE|      -1|      -8|      -9|
|   JFK| CLE|     -17|     -10|     -27|
|   JFK| CLE|     -30|      -9|     -39|
|   JFK| CLE|       3|      -5|      -2|
|   JFK| CLE|      -6|     -10|     -16|
|   JFK| CLE|     -20|      -4|     -24|
|   JFK| CLE|     -12|      -6|     -18|
|   JFK| CLE|       5|      -8|      -3|
|   JFK| IAD|     -15|      -3|     -18|
|   JFK| IAD|      -4|       1|      -3|
|   JFK| IAD|     -13|     -16|     -29|
|   JFK| IAD|      -7|       6|      -1|
|   JFK| CLT|      -8|       0|      -8|
+------+----+--------+--------+--------+
only showing top

53321

In [12]:
#L’altre funció de filtratge disponible es diu filter. És molt similar a la funció where, amb l’a-
#vantatge que es poden fer servir varibles de Python per establir els criteris sobre les condicions de
#filtratge.
from pyspark.sql.functions import col
i = 0
city = "JFK"
df5 = df4.filter(col("SumDelay") < i).filter(col("Origin") == city)
df5.count()

53321

In [13]:
#Exercici
"""
Atesos els exemples que hem vist, ens podem preguntar: donat l’aeroport d’origen de Nova York,
quin és el retard total (columna SumDelay) mínim, màxim i mig associat? Podeu provar també la
funció sum, una transformació que realitzarà la suma de la columna que especifiqueu.
"""
from pyspark.sql.functions import sum
"Dos soluciones diferentes"
df5 = df4.where("Origin == 'JFK'").select(max("SumDelay"),min("SumDelay"),avg("SumDelay")).show()
df5 = df4.where("Origin == 'JFK'").select("SumDelay").groupBy().sum().show()

+-------------+-------------+-----------------+
|max(SumDelay)|min(SumDelay)|    avg(SumDelay)|
+-------------+-------------+-----------------+
|         3111|          -90|35.16175687302823|
+-------------+-------------+-----------------+

+-------------+
|sum(SumDelay)|
+-------------+
|      4290965|
+-------------+



In [14]:
from pyspark.sql.functions import asc, desc
df5 = df4.sort(asc("SumDelay"))
df5.show()
df5 = df4.sort(desc("SumDelay")).limit(5)
df5.show()

+------+----+--------+--------+--------+
|Origin|Dest|ArrDelay|DepDelay|SumDelay|
+------+----+--------+--------+--------+
|   AKN| ANC|    -312|    -305|    -617|
|   CHA| ATL|    -175|    -165|    -340|
|   ANC| FAI|    -162|    -165|    -327|
|   AUS| ATL|    -132|    -124|    -256|
|   ATL| AVL|    -116|    -111|    -227|
|   ANC| SEA|     -13|    -168|    -181|
|   SAN| OKC|    -157|     -19|    -176|
|   HNL| KOA|     -89|     -82|    -171|
|   SFO| HNL|       8|    -169|    -161|
|   ADK| ANC|     -83|     -72|    -155|
|   SJC| SBA|     -82|     -67|    -149|
|   ITO| HNL|     -74|     -71|    -145|
|   ADK| ANC|     -78|     -67|    -145|
|   GNV| ATL|     -82|     -62|    -144|
|   ANC| SEA|      -5|    -137|    -142|
|   PHL| SJU|     -79|     -60|    -139|
|   ADK| ANC|     -79|     -58|    -137|
|   YAK| JNU|     -73|     -64|    -137|
|   KOA| HNL|     -72|     -65|    -137|
|   DHN| ATL|     -75|     -60|    -135|
+------+----+--------+--------+--------+
only showing top

In [15]:
#UNIQUE Elements
df5 = df4.select("Origin").distinct()
df5.count()

304

In [16]:
#Exercici
#Quantes destinacions diferents hi ha? A més, què és el que fa la següent operació?
df4.select("Origin","Dest").distinct().count()
df4.select("Dest").distinct().show()
df4.select("Dest").distinct().count()

+----+
|Dest|
+----+
| BGM|
| PSE|
| DLG|
| INL|
| MSY|
| GEG|
| SNA|
| BUR|
| GRB|
| GTF|
| IDA|
| GRR|
| LWB|
| EUG|
| PSG|
| PVD|
| GSO|
| MYR|
| ISO|
| OAK|
+----+
only showing top 20 rows



304

In [17]:
dades = df4.limit(5).collect()
dades
dades[0]
dades[0][3]

7

In [18]:
#Escriure a disc
df4.rdd.getNumPartitions()
#Per defecte, si ara escrivíssim les dades a disc, es crearien tants fitxers com particions hi ha
df4_one = df4.coalesce(1)
df4_one.rdd.getNumPartitions()
df4_one = spark.write.format("csv") \
.option("header", "true") \
.save("df4_one.csv")

AttributeError: 'SparkSession' object has no attribute 'write'

In [ ]:
##Exercici
"""És interessant que feu un experiment: abans hem ordenat els elements de df4 pel valor de SumDelay.
Intenteu ordenar ara els valors de df4_one. Hi haurà alguna diferència en l’eficiència de l’ordenació?
Feu servir alguna eina gràfica que us mostri el consum de CPU per veure-ho. Per a què és útil tenir
les dades dividides en particions?
"""
